In [8]:
import numpy as np
import random
import math
from operator import itemgetter
import pandas as pd
import sys
import operator
import time

In [5]:
class KPGA:
    def __init__(self, cap, values, weights,P,maxGen): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.parents = []
        self.newparents = []
        self.meilleurs = []
        self.meilleurs_p = [] 
        self.maxGen = maxGen
        self.population = P

    
    # création de la population initiale
    def initialisation(self):

        for i in range(self.population):
            parent = []
            for k in range(0, len(self.weights)):
                k = random.randint(0, 1)
                parent.append(k)
            self.parents.append(parent)
    
    def calcul_fitness(self,sac):
        E=0
        cap=0
        for i in range(len(sac)):
            if sac[i]==1:
                E=E+self.values[i]
                cap=cap+self.weights[i]
        # si la valeur est supérieure à la capacité définie 
        # alors on retoure -1 sinon on retourne
        if cap > self.capacity:
            return -1
        else: 
            return E
    def evaluation(self):
        paire_parent = self.population // 2
        for i in range(len(self.parents)):
            parent = self.parents[i]
            ft = self.calcul_fitness(parent)
            self.meilleurs.append((ft, parent))
        self.meilleurs.sort(key=operator.itemgetter(0), reverse=True)
        #print(self.meilleurs)
        self.meilleurs_p = self.meilleurs[:paire_parent]
        self.meilleurs_p = [x[1] for x in self.meilleurs_p]

    def mutation(self, enfant):

        for i in range(len(enfant)):
            k = random.uniform(0, 1)
            if k > 0.5:
                if enfant[i] == 1:
                    enfant[i] = 0
                else: 
                    enfant[i] = 1
        return enfant
 
    def crossover(self, p1, p2):

        threshold = random.randint(1, len(p1)-1)
        tmp1 = p1[threshold:]
        tmp2 = p2[threshold:]
        p1 = p1[:threshold]
        p2 = p2[:threshold]
        p1.extend(tmp2)
        p2.extend(tmp1)

        return p1, p2
    def new_Generation(self):
        self.evaluation()
        newparents = []
        pop = len(self.meilleurs_p)-1

        sample = random.sample(range(pop), pop)
        for i in range(0, pop):
            # selectionner un index aléatoire du meilleur parents pour rendre le process aléatoire
            if i < pop-1:
                r1 = self.meilleurs_p[i]
                r2 = self.meilleurs_p[i+1]
                nenfant1, nenfant2 = self.crossover(r1, r2)
                newparents.append(nenfant1)
                newparents.append(nenfant2)
            else:
                r1 = self.meilleurs_p[i]
                r2 = self.meilleurs_p[0]
                nenfant1, nenfant2 = self.crossover(r1, r2)
                newparents.append(nenfant1)
                newparents.append(nenfant2)

        # mutation
        for i in range(len(newparents)):
            newparents[i] = self.mutation(newparents[i])
        self.newparents=newparents

    def solveKsack_GA(self):
        for i in range(self.maxGen):
            #print("création d'une nouvelle génération pour la {} fois" .format(i))
            if i!=0:
                self.parents = self.newparents
            self.meilleurs = []
            self.meilleurs_p = []
            self.new_Generation()
        #print(self.newparents)
        self.parents = self.newparents
        #print(self.parents)
        self.evaluation()
        sol=self.meilleurs_p
        #print(sol)
        return sol[0],self.calcul_fitness(sol[0])

In [6]:
P=30
maxGen=60

In [7]:
cap=10
values=[40,50,100,95,30]
weights=[2,3,1,5,3]
kga=KPGA(cap,values,weights,P,maxGen)

In [10]:
start = time.time()
maxiter=20
solutions=[]
for i in range(maxiter):
    kga.initialisation()
    sol,f=kga.solveKsack_GA()
    #print(sol," ",f)
    solutions.append((sol,f))
#print(solutions)
solutions.sort(key=operator.itemgetter(1), reverse=True)
print(solutions[0])
end = time.time()

print("time", end - start)

([0, 1, 1, 1, 0], 245)
time 0.42999696731567383


In [19]:
def chargement_fichier(chemin):
    data = pd.read_csv(chemin)
    cap=int(data.columns[0].split()[1])
    data = pd.read_csv(chemin, header=None, names=(['v','w']),sep=" ")
    v=data.v.values
    w=data.w.values
    return cap,v,w

In [21]:
cap,values,weights=chargement_fichier(r"C:\\Users\\surfaC\\Desktop\\Master SID\\Optimisation Combinatoire\\TP\\TP4\\dataKnapsack\\large_scale\\knapPI_1_100_1000_1")

In [22]:
print('cap:',cap)
print('values:',values)
print('weights:',weights)

cap: 995
values: [100  94 506 416 992 649 237 457 815 446 422 791 359 667 598   7 544 334
 766 994 893 633 131 428 700 617 874 720 419 794 196 997 116 908 539 707
 569 537 931 726 487 772 513  81 943  58 303 764 536 724 789 479 142 339
 641 196 494  66 824 208 711 800 314 289 401 466 689 833 225 244 849 113
 379 361  65 486 686 286 889  24 491 891  90 181 214  17 472 418 419 356
 682 306 201 385 952 500 194 737 324 992 224]
weights: [995 485 326 248 421 322 795  43 845 955 252   9 901 122  94 738 574 715
 882 367 984 299 433 682  72 874 138 856 145 995 529 199 277  97 719 242
 107 122  70  98 600 645 267 972 895 213 748 487 923  29 674 540 554 467
  46 710 553 191 724 730 988  90 340 549 196 865 678 570 936 722 651 123
 431 508 585 853 642 992 725 286 812 859 663  88 179 187 619 261 846 192
 261 514 886 530 849 294 799 391 330 298 790]


In [25]:
P=100
maxGen=200
kga=KPGA(cap,values,weights,P,maxGen)

In [26]:
start = time.time()
maxiter=1000
solutions=[]
for i in range(maxiter):
    kga.initialisation()
    sol,f=kga.solveKsack_GA()
    #print(sol," ",f)
    solutions.append((sol,f))
#print(solutions)
solutions.sort(key=operator.itemgetter(1), reverse=True)
print(solutions[0])
end = time.time()

print("time", end - start)

KeyboardInterrupt: 

In [ ]:
cap,values,weights=chargement_fichier(r"C:\\Users\\surfaC\\Desktop\\Master SID\\Optimisation Combinatoire\\TP\\TP4\\dataKnapsack\\large_scale\\knapPI_3_200_1000_1")

In [ ]:
print('cap:',cap)
print('values:',values)
print('weights:',weights)

In [ ]:
P=100
maxGen=300
kga=KPGA(cap,values,weights,P,maxGen)

In [ ]:
start = time.time()
maxiter=1500
solutions=[]
for i in range(maxiter):
    kga.initialisation()
    sol,f=kga.solveKsack_GA()
    #print(sol," ",f)
    solutions.append((sol,f))
#print(solutions)
solutions.sort(key=operator.itemgetter(1), reverse=True)
print(solutions[0])
end = time.time()

print("time", end - start)

In [20]:
P=60
maxGen=500
cap=878
values=[44,46,90,72,91,40,75,35,8,54,78,40,77,15,61,17,75,29,75,63]
print(len(values))
weights=[92,4,43,83,84,68,92,82,6,44,32,18,56,83,25,96,70,48,14,58]
print(len(weights))
kga=KPGA(cap,values,weights,P,maxGen)
start = time.time()
maxiter=1200
solutions=[]
for i in range(maxiter):
    kga.initialisation()
    sol,f=kga.solveKsack_GA()
    #print(sol," ",f)
    solutions.append((sol,f))
#print(solutions)
solutions.sort(key=operator.itemgetter(1), reverse=True)
print(solutions[0])
end = time.time()

print("time", end - start)

20
20
([1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1], 1004)
time 929.3078801631927


In [16]:
P=50
maxGen=100
cap=269
values=[55,10,47,5,4,50,8,61,85,87]
print(len(values))
weights=[95,4,60,32,23,72,80,62,65,46]
print(len(weights))
kga=KPGA(cap,values,weights,P,maxGen)
start = time.time()
maxiter=150
solutions=[]
for i in range(maxiter):
    kga.initialisation()
    sol,f=kga.solveKsack_GA()
    #print(sol," ",f)
    solutions.append((sol,f))
#print(solutions)
solutions.sort(key=operator.itemgetter(1), reverse=True)
print(solutions[0])
end = time.time()

print("time", end - start)

10
10
([0, 1, 1, 1, 0, 0, 0, 1, 1, 1], 295)
time 10.108218669891357
